In [2]:
#Imports and paths setup
import sys
import rp
import torch
import numpy as np
from einops import rearrange
import shlex
from icecream import ic

top_dir = rp.get_git_toplevel()

ltx_dir = rp.path_join(top_dir, 'LTX2')
nfs_models_dir = rp.path_join(ltx_dir, 'models')

sys.path+=[
    nfs_models_dir,
]

from download_models import local_download_dir, download_from_web
local_models_dir = local_download_dir

models_dir = local_models_dir

# LTX Pipeline imports
from ltx_core.loader import LTXV_LORA_COMFY_RENAMING_MAP, LoraPathStrengthAndSDOps
from ltx_pipelines.ti2vid_two_stages import TI2VidTwoStagesPipeline

# Model paths
checkpoint_path        = rp.path_join(models_dir, "ltx-2-19b-dev.safetensors")
distilled_lora_path    = rp.path_join(models_dir, "ltx-2-19b-distilled-lora-resized_dynamic_fro095_avg_rank_242_bf16.safetensors")
spatial_upsampler_path = rp.path_join(models_dir, "ltx-2-spatial-upscaler-x2-1.0.safetensors")
gemma_root             = models_dir  # Contains text_encoder/ with model files

In [3]:
#Basic Post-Paths/Imports Setup
IN_NOTEBOOK = rp.running_in_jupyter_notebook()
DEVICE = rp.select_torch_device(prefer_used=True, reserve=True) #Select the GPU
DTYPE = torch.bfloat16

download_from_web() #Ensure sure base LTX models are downloaded

print(f"Running in notebook: {IN_NOTEBOOK}")
print(f"Top directory: {top_dir}")

                      ┏━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━┳━━━━━━┳━━━━━━━━━━━┓
                      ┃ GPU ID ┃         Name          ┃      Used      ┃   Free ┃ Total ┃ Temp ┃ Util ┃ Processes ┃
                      ┡━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━╇━━━━━━╇━━━━━━━━━━━┩
Selecting cuda:0 –––> │   0    │ NVIDIA A100-SXM4-80GB │ 694.6MB   0.8% │ 79.3GB │  80GB │ 36°C │   0% │           │
                      │   1    │ NVIDIA A100-SXM4-80GB │ 694.6MB   0.8% │ 79.3GB │  80GB │ 34°C │   0% │           │
                      │   2    │ NVIDIA A100-SXM4-80GB │ 694.6MB   0.8% │ 79.3GB │  80GB │ 35°C │   0% │           │
                      │   3    │ NVIDIA A100-SXM4-80GB │ 694.6MB   0.8% │ 79.3GB │  80GB │ 33°C │   0% │           │
                      │   4    │ NVIDIA A100-SXM4-80GB │ 694.6MB   0.8% │ 79.3GB │  80GB │ 36°C │   0% │           │
                      │   5    │ NVIDIA A100-SXM4-80GB │ 694.6MB

In [29]:
# Helpers
def show_video(video):
    if IN_NOTEBOOK:
        rp.display_video(video)

In [4]:
# T2V (Text-to-Video) Pipeline Setup and Generation
distilled_lora = [
    LoraPathStrengthAndSDOps(
        distilled_lora_path,
        0.6,
        LTXV_LORA_COMFY_RENAMING_MAP,
    ),
]

pipeline = TI2VidTwoStagesPipeline(
    checkpoint_path=checkpoint_path,
    distilled_lora=distilled_lora,
    spatial_upsampler_path=spatial_upsampler_path,
    gemma_root=gemma_root,
    loras=[],
)

In [ ]:
# Generate video from text (T2V - no image conditioning)

drone_prompt = "Drone shot, helicopter flying fast through a narrow rocky canyon, sun-kissed day, clear turquoise water below, white foam waves, motion blur, sharp focus"

ltx_video, ltx_audio = pipeline(
    prompt=drone_prompt,
    negative_prompt="",
    seed=42,
    height=512,
    width=768,
    num_frames=121,
    frame_rate=25.0,
    num_inference_steps=40, #Default 40
    cfg_guidance_scale=3.0,
    images=[],  # Empty list for T2V (text-to-video)
)

with torch.inference_mode():
    video = next(ltx_video)
    video = rp.as_numpy_array(video)

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
show_video(video)